In [0]:
# imports

import json
from utils.api import buscar_dados, obter_estado_id_por_uf
from pyspark.sql.functions import current_timestamp, lit
import pandas as pd
 

In [0]:
#Parametros para facilitar a execução e reutilização do notebook para carga de diferentes anos e estados
dbutils.widgets.text("anos_param", "2023,2022", "Lista de Anos")
dbutils.widgets.text("uf", "MG", "UF")

In [0]:
#Variaveis do Unity Catalog
CATALOGO = "agroclima"
SCHEMA = "bronze"
TABELA = "producao_milho"

# Montar caminho da tabela
caminho_tabela = f"{CATALOGO}.{SCHEMA}.{TABELA}"



In [0]:
# Transforma os parametros em variaveis
lista_anos = dbutils.widgets.get("anos_param").split(",")
uf = dbutils.widgets.get("uf")

In [0]:
# Seta Variavel com o Id da UF 
estado_id = obter_estado_id_por_uf(uf)


In [0]:
# Função que consome a API do IBGE e retorna dados de produção de milho
def obter_producao_milho(ano, estado_id):
       
    # Cria URL com base nos parametros passados
    IBGE_URL = f"https://servicodados.ibge.gov.br/api/v3/agregados/839/periodos/{ano}/variaveis/109|216|214?localidades=N6[N3[{estado_id}]]&classificacao=81[114253,114254]"
    
    # Executa API do IBGE buscando dados da Produção de Milho
    dados_producao_milho = buscar_dados(IBGE_URL)

    df_nomralizado_producao_milho = pd.json_normalize(dados_producao_milho)
    
    #Cria Dataframe com dados do Json
    df_producao_milho = spark.createDataFrame(df_nomralizado_producao_milho)

    #Adiciona colunas para controle de carga
    df_producao_milho = df_producao_milho.withColumn("ano", lit(ano)) \
           .withColumn("uf", lit(uf)) \
           .withColumn("_data_ingestao", current_timestamp())

    return df_producao_milho


In [0]:
for ano in lista_anos:
    try:
        # Chama Função que consome a API com cada paramtero de Ano da Lista passada em parametro
        df_producao_milho = obter_producao_milho(ano, estado_id)

        #Cria Tabela Delta no Unity Catalog  atualizando somente ano e uf passado em parametro
        df_producao_milho.write.format("delta") \
            .mode("overwrite") \
            .partitionBy("uf", "variavel", "ano") \
            .option("replaceWhere", f"ano = {ano} AND uf = '{uf}'") \
            .option("mergeSchema", "true") \
            .saveAsTable(caminho_tabela)

    except Exception as e:
        pass